In [4]:
import pandas as pd
import numpy as np
import nltk
import time
import matplotlib.pyplot as plt
import seaborn as sns
import os, glob
import csv
from functools import reduce
import re
import string


In [88]:
labels = ["Direct Harassment", "Hate Speech","Sexual Harassment","Trolling", "Others", "Toxic"]
col = ["message", "Direct Harassment", "Hate Speech","Sexual Harassment","Trolling", "Others", "Toxic"]
src_folder = "../Annotations/CSVs/"
files = ['usanews.csv', 'foxnews0.csv', 'foxnews1.csv', 'foxnews2.csv', 'foxnews3.csv', 'nogla0.csv','pew0.csv', 'rae0_0.csv', 
         'rae0_1.csv', 'terror0_0.csv', 'terror0_1.csv']
dest_folder = ""


In [89]:
def process(tokens):
    tags = []
    for word, tag in tokens:
        tags.append(tag)
    return " ".join(tags)

In [16]:
for file in files:
    df = pd.read_csv(folder+file)
    df["POS-SEQ"] = df['POS'].apply(lambda tokens: process(eval(tokens)))
    df.to_csv(folder + file)

In [78]:
from sklearn.feature_extraction.text import TfidfVectorizer
def get_tfidf_vect(documents):
    vectorizer = TfidfVectorizer(ngram_range=(1,4), max_df=0.95, analyzer = 'word', min_df=0.05, max_features=10000, lowercase=False)
    vector = vectorizer.fit_transform(documents)
    feature_names = np.array(vectorizer.get_feature_names())
    return pd.DataFrame(vector.todense(), columns=feature_names), vector, feature_names
def get_combined_tfidf(df, features, labels):
    tfidf_df = df
    combined_tfidf = pd.DataFrame(columns= (list(features) + labels))
    for label in labels:
        combined_scores = tfidf_df[tfidf_df[label] == 1][features].sum()
        to_append = list(combined_scores) + [0,0,0,0,0,0]
        row = pd.Series(to_append, index = (list(features) + labels))
        combined_tfidf = combined_tfidf.append(row, ignore_index=True)
        combined_tfidf.loc[combined_tfidf.index[-1],label] = 1.0
    return combined_tfidf

In [79]:
src_folder = "../Annotations/CSVs/"
dest_folder = "combined_tfidf/"
for file in files:
    test = pd.read_csv(folder+file)
    df, vector, features = get_tfidf_vect(test["POS-SEQ"])
    df[labels] = test[labels]
    df[labels] = df[labels].astype("float").replace(r'^\s*$', np.nan, regex=True).fillna(0).replace(np.nan,0)
    df = get_combined_tfidf(df, features, labels)
    #display(df)
    df.to_csv(dest_folder+file)

In [80]:
from scipy.stats import spearmanr
def get_corr_vals(df):
    features = df.columns.difference(labels)
    pvals = {}
    spm = {}
    for label in labels:
        p_values = {}
        sp_coeff = {}
        for f in features:
            print(f)
            sp, pval = spearmanr(df[label], df[f])
            p_values[f] = pval
            sp_coeff[f] = sp
        pvals[label] = p_values
        spm[label] = sp_coeff
        
        x = pd.DataFrame.from_dict(pvals)
        y = pd.DataFrame.from_dict(spm)
    return x, y


In [87]:
src_folder = "combined_tfidf/"
dest_folder = "correlational_analysis/"
prefix_1 = "spr"
prefix_2 = "pval"
for file in files:
    test = pd.read_csv(src_folder+file)
    test = test.drop(['Unnamed: 0'], axis=1)
    pval, spm = get_corr_vals(test)
#     pval.to_csv(dest_folder+file)
    display(pval)

,Direct Harassment,Hate Speech,Sexual Harassment,Trolling,Others,Toxic
ADJ,0.433858,0.277963,0.801909,0.801909,0.277963,0.150200
ADJ NOUN,0.410304,0.410304,0.124880,0.410304,0.410304,0.792664
ADP,0.241982,0.401788,0.401788,0.789282,0.401788,0.241982
ADP DET,NaN,NaN,NaN,NaN,NaN,NaN
ADV,0.410304,0.410304,0.410304,0.792664,0.410304,0.124880
ADV VERB,NaN,NaN,NaN,NaN,NaN,NaN
AUX,0.150200,0.277963,0.801909,0.801909,0.277963,0.433858
AUX PART,0.177808,0.541470,0.541470,0.541470,0.541470,0.177808
AUX PART VERB,0.177808,0.541470,0.541470,0.541470,0.541470,0.177808
AUX PRON,0.704000,0.704000,0.704000,0.000000,0.704000,0.704000


,Direct Harassment,Hate Speech,Sexual Harassment,Trolling,Others,Toxic
ADJ,0.433858,0.277963,0.801909,0.801909,0.277963,0.150200
ADJ NOUN,0.410304,0.410304,0.124880,0.410304,0.410304,0.792664
ADP,0.241982,0.401788,0.401788,0.789282,0.401788,0.241982
ADP DET,NaN,NaN,NaN,NaN,NaN,NaN
ADV,0.410304,0.410304,0.410304,0.792664,0.410304,0.124880
ADV VERB,NaN,NaN,NaN,NaN,NaN,NaN
AUX,0.150200,0.277963,0.801909,0.801909,0.277963,0.433858
AUX PART,0.177808,0.541470,0.541470,0.541470,0.541470,0.177808
AUX PART VERB,0.177808,0.541470,0.541470,0.541470,0.541470,0.177808
AUX PRON,0.704000,0.704000,0.704000,0.000000,0.704000,0.704000


,Direct Harassment,Hate Speech,Sexual Harassment,Trolling,Others,Toxic
ADJ,0.433858,0.277963,0.801909,0.801909,0.277963,0.150200
ADJ NOUN,0.410304,0.410304,0.124880,0.410304,0.410304,0.792664
ADP,0.241982,0.401788,0.401788,0.789282,0.401788,0.241982
ADP DET,NaN,NaN,NaN,NaN,NaN,NaN
ADV,0.410304,0.410304,0.410304,0.792664,0.410304,0.124880
ADV VERB,NaN,NaN,NaN,NaN,NaN,NaN
AUX,0.150200,0.277963,0.801909,0.801909,0.277963,0.433858
AUX PART,0.177808,0.541470,0.541470,0.541470,0.541470,0.177808
AUX PART VERB,0.177808,0.541470,0.541470,0.541470,0.541470,0.177808
AUX PRON,0.704000,0.704000,0.704000,0.000000,0.704000,0.704000


,Direct Harassment,Hate Speech,Sexual Harassment,Trolling,Others,Toxic
ADJ,0.433858,0.277963,0.801909,0.801909,0.277963,0.150200
ADJ NOUN,0.410304,0.410304,0.124880,0.410304,0.410304,0.792664
ADP,0.241982,0.401788,0.401788,0.789282,0.401788,0.241982
ADP DET,NaN,NaN,NaN,NaN,NaN,NaN
ADV,0.410304,0.410304,0.410304,0.792664,0.410304,0.124880
ADV VERB,NaN,NaN,NaN,NaN,NaN,NaN
AUX,0.150200,0.277963,0.801909,0.801909,0.277963,0.433858
AUX PART,0.177808,0.541470,0.541470,0.541470,0.541470,0.177808
AUX PART VERB,0.177808,0.541470,0.541470,0.541470,0.541470,0.177808
AUX PRON,0.704000,0.704000,0.704000,0.000000,0.704000,0.704000


,Direct Harassment,Hate Speech,Sexual Harassment,Trolling,Others,Toxic
ADJ,0.433858,0.277963,0.801909,0.801909,0.277963,0.150200
ADJ NOUN,0.410304,0.410304,0.124880,0.410304,0.410304,0.792664
ADP,0.241982,0.401788,0.401788,0.789282,0.401788,0.241982
ADP DET,NaN,NaN,NaN,NaN,NaN,NaN
ADV,0.410304,0.410304,0.410304,0.792664,0.410304,0.124880
ADV VERB,NaN,NaN,NaN,NaN,NaN,NaN
AUX,0.150200,0.277963,0.801909,0.801909,0.277963,0.433858
AUX PART,0.177808,0.541470,0.541470,0.541470,0.541470,0.177808
AUX PART VERB,0.177808,0.541470,0.541470,0.541470,0.541470,0.177808
AUX PRON,0.704000,0.704000,0.704000,0.000000,0.704000,0.704000


...

,Direct Harassment,Hate Speech,Sexual Harassment,Trolling,Others,Toxic
ADJ,0.433858,0.277963,0.801909,0.801909,0.277963,0.150200
ADJ NOUN,0.410304,0.410304,0.124880,0.410304,0.410304,0.792664
ADP,0.241982,0.401788,0.401788,0.789282,0.401788,0.241982
ADP DET,NaN,NaN,NaN,NaN,NaN,NaN
ADV,0.410304,0.410304,0.410304,0.792664,0.410304,0.124880
ADV VERB,NaN,NaN,NaN,NaN,NaN,NaN
AUX,0.150200,0.277963,0.801909,0.801909,0.277963,0.433858
AUX PART,0.177808,0.541470,0.541470,0.541470,0.541470,0.177808
AUX PART VERB,0.177808,0.541470,0.541470,0.541470,0.541470,0.177808
AUX PRON,0.704000,0.704000,0.704000,0.000000,0.704000,0.704000


,Direct Harassment,Hate Speech,Sexual Harassment,Trolling,Others,Toxic
ADJ,0.433858,0.277963,0.801909,0.801909,0.277963,0.150200
ADJ NOUN,0.410304,0.410304,0.124880,0.410304,0.410304,0.792664
ADP,0.241982,0.401788,0.401788,0.789282,0.401788,0.241982
ADP DET,NaN,NaN,NaN,NaN,NaN,NaN
ADV,0.410304,0.410304,0.410304,0.792664,0.410304,0.124880
ADV VERB,NaN,NaN,NaN,NaN,NaN,NaN
AUX,0.150200,0.277963,0.801909,0.801909,0.277963,0.433858
AUX PART,0.177808,0.541470,0.541470,0.541470,0.541470,0.177808
AUX PART VERB,0.177808,0.541470,0.541470,0.541470,0.541470,0.177808
AUX PRON,0.704000,0.704000,0.704000,0.000000,0.704000,0.704000


,Direct Harassment,Hate Speech,Sexual Harassment,Trolling,Others,Toxic
ADJ,0.433858,0.277963,0.801909,0.801909,0.277963,0.150200
ADJ NOUN,0.410304,0.410304,0.124880,0.410304,0.410304,0.792664
ADP,0.241982,0.401788,0.401788,0.789282,0.401788,0.241982
ADP DET,NaN,NaN,NaN,NaN,NaN,NaN
ADV,0.410304,0.410304,0.410304,0.792664,0.410304,0.124880
ADV VERB,NaN,NaN,NaN,NaN,NaN,NaN
AUX,0.150200,0.277963,0.801909,0.801909,0.277963,0.433858
AUX PART,0.177808,0.541470,0.541470,0.541470,0.541470,0.177808
AUX PART VERB,0.177808,0.541470,0.541470,0.541470,0.541470,0.177808
AUX PRON,0.704000,0.704000,0.704000,0.000000,0.704000,0.704000


,Direct Harassment,Hate Speech,Sexual Harassment,Trolling,Others,Toxic
ADJ,0.433858,0.277963,0.801909,0.801909,0.277963,0.150200
ADJ NOUN,0.410304,0.410304,0.124880,0.410304,0.410304,0.792664
ADP,0.241982,0.401788,0.401788,0.789282,0.401788,0.241982
ADP DET,NaN,NaN,NaN,NaN,NaN,NaN
ADV,0.410304,0.410304,0.410304,0.792664,0.410304,0.124880
ADV VERB,NaN,NaN,NaN,NaN,NaN,NaN
AUX,0.150200,0.277963,0.801909,0.801909,0.277963,0.433858
AUX PART,0.177808,0.541470,0.541470,0.541470,0.541470,0.177808
AUX PART VERB,0.177808,0.541470,0.541470,0.541470,0.541470,0.177808
AUX PRON,0.704000,0.704000,0.704000,0.000000,0.704000,0.704000


,Direct Harassment,Hate Speech,Sexual Harassment,Trolling,Others,Toxic
ADJ,0.433858,0.277963,0.801909,0.801909,0.277963,0.150200
ADJ NOUN,0.410304,0.410304,0.124880,0.410304,0.410304,0.792664
ADP,0.241982,0.401788,0.401788,0.789282,0.401788,0.241982
ADP DET,NaN,NaN,NaN,NaN,NaN,NaN
ADV,0.410304,0.410304,0.410304,0.792664,0.410304,0.124880
ADV VERB,NaN,NaN,NaN,NaN,NaN,NaN
AUX,0.150200,0.277963,0.801909,0.801909,0.277963,0.433858
AUX PART,0.177808,0.541470,0.541470,0.541470,0.541470,0.177808
AUX PART VERB,0.177808,0.541470,0.541470,0.541470,0.541470,0.177808
AUX PRON,0.704000,0.704000,0.704000,0.000000,0.704000,0.704000
